In [1]:
from flask import Flask, request, send_file
import numpy as np
import tensorflow as tf
import cv2
import os

app = Flask(__name__)
model = tf.keras.models.load_model("animals.keras")  # 2-class model

# Constants
CONFIDENCE_THRESHOLD = 0.8  # Fixed threshold
CLASS_NAMES = ['honeybadger', 'leopard']

# Global state
last_prediction = "No prediction made yet."
last_confidence = 0.0
detection_counts = {'honeybadger': 0, 'leopard': 0, 'false_detection': 0}
last_image_path = "last_image.jpg"  # Temp file for last image

def preprocess_and_save(img_bytes):
    """Process uploaded image and save a copy"""
    with open(last_image_path, 'wb') as f:
        f.write(img_bytes)
    
    nparr = np.frombuffer(img_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (240, 240))
    img = img.astype(np.float32) / 255.0
    return img

def predict_image(image):
    """Make prediction with fixed threshold"""
    img_array = np.expand_dims(image, axis=0)
    pred_probs = model.predict(img_array, verbose=0)[0]
    max_prob = np.max(pred_probs)
    predicted_class = np.argmax(pred_probs)
    
    if max_prob < CONFIDENCE_THRESHOLD:
        return "false_detection", max_prob
    return CLASS_NAMES[predicted_class], max_prob

@app.route('/')
def home():
    count_display = "<br>".join([f"{cls}: {count}" for cls, count in detection_counts.items()])
    image_html = f'<img src="/last_image" width="300">' if os.path.exists(last_image_path) else "No image received yet."
    
    return f"""
    <h2>Animal Classifier</h2>
    <p><strong>Last prediction:</strong> {last_prediction} (confidence: {last_confidence:.2f})</p>
    <p><strong>Detection counts:</strong><br>{count_display}</p>
    <p><strong>Last image:</strong><br>{image_html}</p>
    <p><em>Using fixed confidence threshold: {CONFIDENCE_THRESHOLD}</em></p>
    """

@app.route('/classify', methods=['POST'])
def classify():
    global last_prediction, last_confidence, detection_counts
    
    if not request.data:
        return "No image data received", 400
    
    try:
        img = preprocess_and_save(request.data)
        label, confidence = predict_image(img)
        
        last_prediction = label
        last_confidence = confidence
        detection_counts[label] += 1
        
        return f"{label} (confidence: {confidence:.2f})"
    except Exception as e:
        return f"Error processing image: {str(e)}", 500

@app.route('/last_image')
def last_image():
    if os.path.exists(last_image_path):
        return send_file(last_image_path, mimetype='image/jpeg')
    return "No image available", 404

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.5:5000
Press CTRL+C to quit
192.168.0.126 - - [14/May/2025 11:36:18] "POST /classify HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:23] "GET / HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:23] "GET /last_image HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:24] "GET /favicon.ico HTTP/1.1" 404 -
192.168.0.126 - - [14/May/2025 11:36:24] "POST /classify HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:29] "GET / HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:29] "GET /last_image HTTP/1.1" 200 -
192.168.0.126 - - [14/May/2025 11:36:30] "POST /classify HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:30] "GET / HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:30] "GET /last_image HTTP/1.1" 304 -
192.168.0.126 - - [14/May/2025 11:36:35] "POST /classify HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:37] "GET / HTTP/1.1" 200 -
192.168.0.5 - - [14/May/2025 11:36:37]